<a href="https://colab.research.google.com/github/kop57890/acerml/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/acerml/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/acerml/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f7c096cb6a0>)

In [3]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [5]:
datas = pd.concat([train_df, test_df], axis = 0, ignore_index=True)
datas = datas.drop(["PassengerId", "Survived"], axis = 1)

In [6]:
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [7]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [8]:
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [9]:
def cabinflow(c):
  if pd.isna(c):
    return c
  else:
    return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabinflow)
datas["Cabin"].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: Cabin, dtype: int64

In [10]:
dic = datas["Ticket"].value_counts()
def ticketflow(t):
  return dic[t]
datas["Ticket"] = datas["Ticket"].apply(ticketflow)
datas["Ticket"].value_counts()

1     713
2     264
3     147
4      64
7      35
5      35
6      24
8      16
11     11
Name: Ticket, dtype: int64

In [11]:
def nameflow(n):
  mid = n.split(".")[0].split(",")[-1]
  return mid.strip()
middle = datas["Name"].apply(nameflow).value_counts()
whitelist = middle.index[:4]

def nameflow2(n):
  mid = nameflow(n)
  if mid in whitelist:
    return mid
  else:
    return None
datas["Name"] = datas["Name"].apply(nameflow2)
datas["Name"].value_counts()

Mr        757
Miss      260
Mrs       197
Master     61
Name: Name, dtype: int64

In [ ]:
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])

In [15]:
datas["Family"] = datas["SibSp"] + datas["Parch"]

In [21]:
x_train = datas.iloc[:len(train_df)]
x_test = datas.iloc[len(train_df):]
y_train = train_df["Survived"]

In [23]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=20, max_depth=5)
scores = cross_val_score(clf, x_train, y_train, cv = 10, n_jobs=-1)
print("10 times", scores)
print("average", np.average(scores))

10 times [0.83333333 0.87640449 0.7752809  0.88764045 0.84269663 0.82022472
 0.84269663 0.79775281 0.87640449 0.84269663]
average 0.8395131086142322
